<a href="https://colab.research.google.com/github/Belaschich/Projeto_Final_SoulCode/blob/main/ProjetoFinal_(Spark_Oficial)_19_11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# IMPORT BIBLIOTECAS E ACESSO GCP

In [ ]:
pip install gcsfs

     |████████████████████████████████| 132 kB 7.5 MB/s 
     |████████████████████████████████| 1.1 MB 46.6 MB/s 
     |████████████████████████████████| 160 kB 47.8 MB/s 
     |████████████████████████████████| 271 kB 67.3 MB/s 
     |████████████████████████████████| 192 kB 62.4 MB/s 


In [ ]:
#BIBLIOTECA PARA CHAVE DE ACESSO GCP

import os

In [ ]:
#IMPORT PANDAS E STORAGE

import pandas as pd
from google.cloud import storage

In [ ]:
#IMPORTA BASE DE ACESSO GCP

service_account_key = r"/content/chaveprogrupo10-332518-4db3212adcd1.json"

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = service_account_key

In [ ]:
pip install pyspark

     |████████████████████████████████| 281.3 MB 38 kB/s 
     |████████████████████████████████| 198 kB 54.5 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.0-py2.py3-none-any.whl size=281805912 sha256=ac26d128c88be52272acdf34b0ed965a29b2123e9db2a13300bfaa3f624b5905
  Stored in directory: /root/.cache/pip/wheels/0b/de/d2/9be5d59d7331c6c2a7c1b6d1a4f463ce107332b1ecd4e80718
Successfully built pyspark


In [ ]:
#IMPORT BIBLIOTECAS SPARK

from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.window import Window
from pyspark.sql.types import StructField, StructType, StringType, IntegerType, DoubleType, DateType


In [ ]:
#PREPARAÇÃO SPARK SESSION

spark = (SparkSession.builder
        .master("local")
        .appName("novo-dataframes")
        .config("spark.ui.port", "4050")
        .getOrCreate())

In [ ]:
#IMPORTAÇÃO BASE "AMERICA DO NORTE" SALVO NO GCP

df1 = pd.read_csv('gs://bucket-projeto-g10/saida_tratado/nivel_pandera/Base_ConsumoNA (2.0).csv')

# ALTERAÇÕES PANDAS


In [ ]:
#RENOMEIA A COLUNA "ESTADO" PARA "UF"

df1 = df1.rename({'Estado': 'UF'}, axis = 1)

In [ ]:
#CRIA UMA COLUNA "CLIMA" COM VALORES NaN

df1['Clima'] = 'NaN'
df1

In [ ]:
#INSERE DADOS NA COLUNA "CLIMA" COM CONDIÇÕES

for index, row in df1.iterrows():
    if row['UF'] == 'Alaska':
        df1.loc[index,'Clima'] =  str('Polar')
   
        df1.loc[index,'Clima'] =  str('Seco')
    elif row['UF'] == 'Connecticut' or row['UF'] == 'Massachusetts'or row['UF'] == 'Minnesota' or row['UF'] == 'Dakota do Norte':
        df1.loc[index,'Clima'] =  str('Continental' elif row['UF'] == 'Colorado':)
    elif row['UF'] == 'Delaware' or row['UF'] == 'Florida' or row['UF'] == 'Illinois' or row['UF'] == 'Kentucky' or row['UF'] ==  'Missouri'or row['UF'] ==  'Tennessee' or row['UF'] ==  'Texas' or row['UF'] ==  'Iowa':
        df1.loc[index,'Clima'] =  str('Subtropical') 

df1

In [ ]:
#VALIDAÇÃO DA COLUNA "CLIMA"

pd.unique(df1['Clima'])

In [ ]:
#VERIFICA DADOS UNICOS DA COLUNA "UF"

pd.unique(df1['UF'])

In [ ]:
#CRIA UMA COLUNA "ESTAÇÃO DO ANO" COM VALORES NaN

df1['EstacaoDoAno'] = 'NaN'
df1

In [ ]:
#INSERE DADOS NA COLUNA "ESTAÇÃO DO ANO" COM CONDIÇÕES

for index, row in df1.iterrows():
    if row['Mes'] == 6 or row['Mes'] == 7 or row['Mes'] == 8:
        df1.loc[index,'EstacaoDoAno'] =  str('Verão')
    elif row['Mes'] == 12 or row['Mes'] == 1 or row['Mes'] == 2:
        df1.loc[index,'EstacaoDoAno'] =  str('Inverno')
    elif row['Mes'] == 3 or row['Mes'] == 4 or row['Mes'] == 5:
        df1.loc[index,'EstacaoDoAno'] =  str('Primavera') 
    elif row['Mes'] == 9 or row['Mes'] == 10 or row['Mes'] == 11:
        df1.loc[index,'EstacaoDoAno'] =  str('Outono') 

df1

In [ ]:
#RENOMEIA OS DADOS DA COLUNA "UF"

df1['UF'].replace('Alaska', 'AK', inplace=True)
df1['UF'].replace('Colorado', 'CO', inplace=True)
df1['UF'].replace('Connecticut', 'CT', inplace=True)
df1['UF'].replace('Delaware', 'DE', inplace=True)
df1['UF'].replace('Florida', 'FL', inplace=True)
df1['UF'].replace('Illinois', 'IL', inplace=True)
df1['UF'].replace('Kentucky', 'KY', inplace=True)
df1['UF'].replace('Massachusetts', 'MA', inplace=True)
df1['UF'].replace('Minnesota', 'MN', inplace=True)
df1['UF'].replace('Missouri', 'MO', inplace=True)
df1['UF'].replace('Dakota do Norte', 'ND', inplace=True)
df1['UF'].replace('Tennessee', 'TN', inplace=True)
df1['UF'].replace('Texas', 'TX', inplace=True)

# NÍVEL SPARK (TRATATIVA)

In [ ]:
#CRIAÇÃO DO "STRUCTURED TYPE" PARA OS DADOS DO DATAFRAME "AMÉRICA DO NORTE"

customSchema = StructType([
    StructField("Ano", IntegerType(), True),
    StructField("Mes", IntegerType(), True),
    StructField("MesRef", StringType(), True),
    StructField("UF", StringType(), True),
    StructField("Categoria", StringType(), True),
    StructField("Galoes", IntegerType(), True),
    StructField("Etanol", IntegerType(), True),
    StructField("Populacao", IntegerType(), True),
    StructField("EtanolPerCapita", DoubleType(), True),
    StructField("Clima", StringType(), True),
    StructField("EstacaoDoAno", StringType(), True),
])

In [ ]:
#CRIA UM DATAFRAME SPARK, COM BASE NOS DADOS DO DF "AN" CRIADO EM PANDAS

df1_spark = spark.createDataFrame(df1, schema=customSchema) 

In [ ]:
#EXIBE OS DADOS DO DF CONVERTIDO

df1_spark.show()

In [ ]:
df1_spark.printSchema()

In [ ]:
#CRIA UM RANK SOBRE A QUANTIDADE DE ETANOL VENDIDO

w0 = Window.orderBy(F.col("Etanol"))
df2 = df1_spark.withColumn("rank", F.row_number().over(w0))
df2.select(F.col("Ano"), F.col("Mes"), F.col("UF"), F.col("Etanol") , F.col('Populacao'),
 F.col("rank")).show(10)

In [ ]:
#CRIA UMA COLUNA DE GALÕES PER CAPITA (GALÕES / POPULAÇÃO)

df2 = df1_spark.withColumn("GaloesPerCapita", F.col("Galoes") / F.col("Populacao"))
df2.show()

# INSIGHTS SPARK

In [ ]:
#CRIA UM AGRUPAMENTO DE GALOES PER CAPITA POR "UF"

group1 = (df2.groupBy(F.col("UF")).agg(
            F.sum("GaloesPerCapita").alias("Soma_GaloesPC"),
            F.avg("GaloesPerCapita").alias("Med_GaloesPC")
            )
).orderBy(F.sum('GaloesPerCapita').desc())

group1.show()

#INSIGHTS:
#DAKOTA DO NORTE APRESENTOU MAIOR CONSUMO (MÉDIA DE 1 GALÃO PER CAPITA)
#KENTUCKY APRESENTOU MENOR CONSUMO (0.7 GALÕES PER CAPITA)

In [ ]:
#CRIA UM AGRUPAMENTO DE GALOES PER CAPITA POR "MESREF" E "UF"

group2 = (df2.groupBy(F.col("MesRef"), F.col("UF")).agg(
            F.sum("GaloesPerCapita").alias("Soma_GaloesPC"),
            F.avg("GaloesPerCapita").alias("Med_GaloesPC")
            )
).orderBy(F.col('MesRef').desc())

group2.show(1000)

#INSIGHT:
#PODE SER USADO PARA CRIAR UM ESTUDO DE PERÍODO COM FILTROS DE ANO E UF

In [ ]:
#CRIA UM RANK SOBRE A QUANTIDADE DE "GALÕES PER CAPITA" POR PERÍODO E "UF"

w1 = Window.orderBy(F.col("GaloesPerCapita").desc())

df2_rank = df2.withColumn("rank", F.row_number().over(w1))
df2_rank.select(F.col("Ano"), F.col("Mes"), F.col("UF"), F.col("GaloesPerCapita"), 
 F.col("rank")).show(1000)

#GERA UM INSIGHT RÁPIDO SOBRE MAIOR CONSUMO POR PERÍODO E UF

In [ ]:
#CRIA UM RANK SOBRE A QUANTIDADE DE "ETANOL PER CAPITA" VENDIDO

w1 = Window.orderBy(F.col("EtanolPerCapita").desc())
df2_rank = df2.withColumn("rank", F.row_number().over(w0))
df2_rank.select(F.col("Ano"), F.col("Mes"), F.col("UF"), F.col("EtanolPerCapita"), 
 F.col("rank")).show(1000)


#GERA UM INSIGHT RÁPIDO SOBRE MAIOR CONSUMO POR PERÍODO E UF NA VISÃO ETANOL

In [ ]:
#CRIA UM RANK SOBRE A QUANTIDADE DE "GALÕES PER CAPITA" VENDIDO POR TIPO DE CLIMA

(df2.groupBy(F.col("Clima")).agg(
            F.sum("GaloesPerCapita").alias("Sum_GaloesPerCapita"),
            ).show()
)

#INSIGHTS:
#MAIOR CONSUMO NAS REGIOES SUBTROPICAIS
#MENOR CONSUMO NAS REGIOES POLAR

# SPARK SQL

In [ ]:
#CRIANDO UMA TABELA A PARTIR DO PYSPARK 

df2.createOrReplaceTempView("Tab_AN")

In [ ]:
#EXIBE EA TABELA CRIADA "Tab_AN"

spark.sql("SELECT * FROM Tab_AN").show()

In [ ]:
#AJUSTANDO O MESREF E CRIANDO UMA NOVA TABELA MAIS FUNCIONAL "Tab_AN_Novo"

spark.sql(
''' 
SELECT 
CASE WHEN CHARACTER_LENGTH(MesRef) == 5 THEN CONCAT('0', MesRef)
ELSE MesRef
END AS MesRefNovo,
UF,
Categoria,
Galoes,
Etanol,
Populacao,
EtanolPerCapita,
Clima,
EstacaoDoAno,
GaloesPerCapita
FROM Tab_AN 
'''
          ).createOrReplaceTempView("Tab_AN_Novo")

In [ ]:
#CRIANDO UM INSIGHT DE CONSUMO POR POPULAÇÃO E ESTADO

spark.sql(
''' 
SELECT 
UF,
ROUND(SUM(Populacao),2) AS SUM_POPULACAO,
ROUND(SUM(EtanolPerCapita),2) AS SUM_ETANOLPC,
ROUND(SUM(GaloesPerCapita),2) AS SUM_GALOESPC,
ROUND(AVG(GaloesPerCapita),2) AS MED_GALOESPC
FROM Tab_AN_Novo
GROUP BY UF
ORDER BY UF
'''
          ).show(1000000)

#INSIGHTS:
#MAIOR CONSUMO: DAKOTA DO NORTE (GALÕES PC) E DELAWARE (ETANOL PC)
#MENOR CONSUMO: KENTUCKY (GALÕES E ETANOL PC)

In [ ]:
# CRIANDO UM INSIGHT DE SOMA  DE GALÕES POR CATEGORIA E ESTADO
spark.sql(
''' 
SELECT 
UF,
CATEGORIA,
ROUND(SUM(GaloesPerCapita),2) AS SUM_GALOESPC
FROM Tab_AN_Novo
GROUP BY UF, CATEGORIA
ORDER BY UF, CATEGORIA
'''
          ).show(1000000)

#INSIGHTS:
#MAIOR CONSUMIDOR DE CERVEJA: DAKOTA DO NORTE
#MAIOR CONSUMIDOR DE DESTILADOS: DELAWARE
#MAIOR CONSUMIDOR DE VINHOS: DELAWARE

In [ ]:
#CRIANDO INSIGHT DE CONSUMO POR ANO E SEMESTRE

spark.sql(
''' 
SELECT 
RIGHT(MesRefNovo, 4) AS ANO,
CASE WHEN LEFT(MesRefNovo, 2) Between 01 and 06 THEN 'Primeiro Semestre'
ELSE 'Segundo Semestre'
END AS Semestre,
ROUND(SUM(EtanolPerCapita),2) AS SUM_ETANOLPC,
ROUND(SUM(GaloesPerCapita),2) AS SUM_GALOESPC,
ROUND(AVG(GaloesPerCapita),2) AS MED_GALOESPC
FROM Tab_AN_Novo 
GROUP BY ANO, Semestre 
ORDER BY ANO, Semestre
'''
          ).show(10000)

#INSIGHTS:
#MAIOR CONSUMO NO SEGUNDO SEMESTRE DE 2020
#MENOR CONSUMO NO PRIMEIRO SEMESTRE DE 2019 E 2020 
#OBS: DESCONSIDERAMOS 2021 POR TER APENAS 2 MESES DE COMPARATIVO

In [ ]:
#CRIANDO INSIGHT DE CONSUMO POR ESTAÇÃO DO ANO
spark.sql(
''' 
SELECT 
RIGHT(MesRefNovo, 4) AS ANO,
EstacaoDoAno,
ROUND(SUM(EtanolPerCapita),2) AS SUM_ETANOLPC,
ROUND(SUM(GaloesPerCapita),2) AS SUM_GALOESPC,
ROUND(AVG(GaloesPerCapita),2) AS MED_GALOESPC
FROM Tab_AN_Novo 
GROUP BY ANO, EstacaoDoAno 
ORDER BY ANO, EstacaoDoAno
'''
          ).show(10000)

#INSIGHTS:
#MAIOR CONSUMO GERAL NO VERÃO 
#MENOR CONSUMO GERAL NO INVERNO

In [ ]:
#CRIANDO INSIGHT DE CONSUMO POR PRODUTO EM CADA ÉPOCA DO ANO
spark.sql(
''' 
SELECT 
EstacaoDoAno,
Categoria,
ROUND(SUM(GaloesPerCapita),2) AS SUM_GALOESPC
FROM Tab_AN_Novo
GROUP BY EstacaoDoAno, Categoria
ORDER BY EstacaoDoAno, Categoria
'''
          ).show(10000)

#INSIGHTS:
# MAIOR CONSUMO DE CERVEJA DURANTE VERÃO
# MAIOR CONSUMO DE DESTILADO DURANTE O INVERNO
# MAIOR CONSUMO DE VINHO DURANTE O INVERNO

# PLOTAGEM PANDAS


In [ ]:
#CONVERTE O DF DE SPARK PARA PANDAS QUE SERÁ USADO PARA AS PLOTAGENS

df2_plot = df2.toPandas()

In [ ]:
g2 = df2_plot.groupby(['Ano', 'Mes'])['GaloesPerCapita'].agg('mean')

In [ ]:
g2.plot(x="Ano", y=["GaloesPerCapita"], kind="bar",figsize=(20,8))


#INSIGHTS:
#MAIOR CONSUMO GERAL ENTRE MAIO E AGOSTO
#MENOR CONSUMO GERAL ENTRE JANEIRO E FEVEREIRO

In [ ]:
g3 = df2_plot.groupby(['Ano', 'Mes'])['EtanolPerCapita'].agg('mean')

In [ ]:
g3.plot(x="Ano", y=["EtanolPerCapita"], kind="bar",figsize=(20,8))

#INSIGHTS:
#COMPORTAMENTO SEMELHANTE

In [ ]:
filtro_ano = df2_plot.Ano == 2020

df3_plot = df2_plot.loc[filtro_ano]

g3 = df3_plot.groupby(['UF'])['EtanolPerCapita'].agg('mean')

In [ ]:
g3.plot(x="UF", y=["EtanolPerCapita"], kind="line",figsize=(20,8))

#INSIGHTS:
#EM 2020 DELAWARE FOI O MAIOR CONSUMIDOR
#KENTUCKY FOI O MENOR CONSUMIDOR NESTE MESMO ANO


In [ ]:
filtro_ano = df2_plot.Ano == 2020

df4_plot = df2_plot.loc[filtro_ano]

g4 = df4_plot.groupby(['UF'])['GaloesPerCapita'].agg('mean')

In [ ]:
g4.plot(x="UF", y=["GaloesPerCapita"], kind="barh",figsize=(20,8))

#INSIGHTS:
#EM 2020 TIVEMOS KENTUCKY E CONNECTICUT FORAM OS MENORES CONSUMIDORES
#COM VALORES PRÓXIMOS ENTRE SI

# EXPORTAÇÃO

In [ ]:
#df2.toPandas().to_csv('gs://bucket-projeto-g10/saida_tratado/nivel_spark/Base_ConsumoNA (3.0).csv', index=False)

In [ ]:
#df2.printSchema()